<a href="https://colab.research.google.com/github/nitish-01/optimisation/blob/main/SCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cplex
!pip install docplex
!pip install cylp
!pip install osmnx
!pip install networkx
from itertools import product
import numpy as np
import cvxpy as cp
import pandas as pd
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
class SCO:
    '''a Model class that solves the supply chain transportation optimization problem.'''

    def __init__(self, framework='CVXPY'):
        # parameters
        self.port_Space = None
        self.date_Space = None
        self.goods_Display = None
        self.index_Port = None
        self.port_Index = None
        self.max_Date = None
        self.min_Date = None
        self.tran_Cost = None
        self.tran_Fixed_Cost = None
        self.tran_Time = None
        self.container_Vol = None
        self.wh_Cost = None
        self.k_Vol = None
        self.k_Value = None
        self.k_DDL = None
        self.k_Start_Port = None
        self.k_End_Port = None
        self.k_Start_Time = None
        self.tax_Pct = None
        self.transit_Duty = None
        self.route_num = None
        self.available_routes = None
        # decision variables
        self.var = None
        self.x = None
        self.var_2 = None
        self.y = None
        self.var_3 = None
        self.z = None
        # result & solution
        self.xs = None
        self.ys = None
        self.zs = None
        self.wh_Cost_Final = None
        self.transport_Cost = None
        self.tax_Cost = None
        self.solution_ = None
        self.arrTime_ = None
        self.objective_value = None
        # helping variables
        self.var_location = None
        self.var_2_location = None
        self.var_3_location = None



    def set_param(self, route, order):
        '''set model parameters based on the read-in route and order information.'''

        bigM = 100000
        route = route[route['Feasibility'] == 1]
        route['Warehouse Cost'][route['Warehouse Cost'].isnull()] = bigM
        route = route.reset_index()

        portSet = set(route['Source']) | set(route['Destination'])

        self.port_Space = len(portSet)
        self.port_Index = dict(zip(range(len(portSet)), portSet))
        self.index_Port = dict(zip(self.port_Index.values(), self.port_Index.keys()))

        self.max_Date = np.max(order['Required Delivery Date'])
        self.min_Date = np.min(order['Order Date'])
        self.date_Space = (self.max_Date - self.min_Date).days
        startWeekday = self.min_Date.weekday() + 1
        weekday = np.mod((np.arange(self.date_Space) + startWeekday), 7)
        weekday[weekday == 0] = 7
        weekdayDateList = {i: [] for i in range(1, 8)}
        for i in range(len(weekday)):
            weekdayDateList[weekday[i]].append(i)
        for i in weekdayDateList:
            weekdayDateList[i] = json.dumps(weekdayDateList[i])

        source = list(route['Source'].replace(self.index_Port))
        destination = list(route['Destination'].replace(self.index_Port))
        DateList = list(route['Weekday'].replace(weekdayDateList).apply(json.loads))

        self.goods_Display = order.shape[0]
        self.tran_Cost = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM
        self.tran_Fixed_Cost = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM
        self.tran_Time = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM

        for i in range(route.shape[0]):
            self.tran_Cost[source[i], destination[i], DateList[i]] = route['Cost'][i]
            self.tran_Fixed_Cost[source[i], destination[i], DateList[i]] = route['Fixed Freight Cost'][i]
            self.tran_Time[source[i], destination[i], DateList[i]] = route['Time'][i]

        self.transit_Duty = np.ones([self.port_Space, self.port_Space]) * bigM
        self.transit_Duty[source, destination] = route['Transit Duty']

        # make the container size of infeasible routes to be small enough, similar to bigM
        self.container_Vol = np.ones([self.port_Space, self.port_Space]) * 0.1
        self.container_Vol[source, destination] = route['Container Size']
        self.container_Vol = self.container_Vol.reshape(self.port_Space, self.port_Space, 1)
        self.wh_Cost = route[['Source', 'Warehouse Cost']].drop_duplicates()
        self.wh_Cost['index'] = self.wh_Cost['Source'].replace(self.index_Port)
        self.wh_Cost = np.array(self.wh_Cost.sort_values(by='index')['Warehouse Cost'])
        self.k_Vol = np.array(order['Volume'])
        self.k_Value = np.array(order['Order Value'])
        self.k_DDL = np.array((order['Required Delivery Date'] - self.min_Date).dt.days)
        self.k_Start_Port = np.array(order['Ship From'].replace(self.index_Port))
        self.k_End_Port = np.array(order['Ship To'].replace(self.index_Port))
        self.k_Start_Time = np.array((order['Order Date'] - self.min_Date).dt.days)
        self.tax_Pct = np.array(order['Tax Percentage'])

        # add available route indexes
        self.route_num = route[['Source', 'Destination']].drop_duplicates().shape[0]
        routes = route[['Source', 'Destination']].drop_duplicates().replace(self.index_Port)
        self.available_routes = list(zip(routes['Source'], routes['Destination']))
        # localization variables of decision variables in the matrix
        var_location = product(self.available_routes, range(self.date_Space), range(self.goods_Display))
        var_location = [(i[0][0], i[0][1], i[1], i[2]) for i in var_location]
        self.var_location = tuple(zip(*var_location))

        var_2_location = product(self.available_routes, range(self.date_Space))
        var_2_location = [(i[0][0], i[0][1], i[1]) for i in var_2_location]
        self.var_2_location = tuple(zip(*var_2_location))

        self.var_3_location = self.var_2_location

    def build_model(self):
        '''overall function to build up model objective and constraints'''
        self.cvxpy_build_model()
        

    def cvxpy_build_model(self):
        '''build up the mathematical programming model's objective and constraints using CVXPY framework.'''

        # 4 dimensional binary decision variable matrix
        self.var = cp.Variable(self.route_num * self.date_Space * self.goods_Display, boolean=True, name='x')
        self.x = np.zeros((self.port_Space, self.port_Space, self.date_Space, self.goods_Display)).astype('object')
        self.x[self.var_location] = list(self.var)
        # 3 dimensional container number matrix
        self.var_2 = cp.Variable(self.route_num * self.date_Space, integer=True, name='y')
        self.y = np.zeros((self.port_Space, self.port_Space, self.date_Space)).astype('object')
        self.y[self.var_2_location] = list(self.var_2)
        # 3 dimensional route usage matrix
        self.var_3 = cp.Variable(self.route_num * self.date_Space, boolean=True, name='z')
        self.z = np.zeros((self.port_Space, self.port_Space, self.date_Space)).astype('object')
        self.z[self.var_3_location] = list(self.var_3)
        # warehouse related cost
        warehouseCost, arrTime, stayTime = self.warehouse_fee(self.x)
        ###objective###
        transport_Cost = np.sum(self.y * self.tran_Cost) + np.sum(self.z * self.tran_Fixed_Cost)
        transit_DutyCost = np.sum(np.sum(np.dot(self.x, self.k_Value), axis=2) * self.transit_Duty)
        tax_Cost = np.sum(self.tax_Pct * self.k_Value) + transit_DutyCost
        objective = cp.Minimize(transport_Cost + warehouseCost + tax_Cost)
        ###constraint###
        constraints = []
        # 1.goods_Display must be shipped out from its origin to another node and shipped to its destination.
        constraints += [np.sum(self.x[self.k_Start_Port[k], :, :, k]) == 1 for k in range(self.goods_Display)]
        constraints += [np.sum(self.x[:, self.k_End_Port[k], :, k]) == 1 for k in range(self.goods_Display)]
        # 2.For each goods_Display k, it couldn't be shipped out from its destination or shipped to its origin.
        constraints += [np.sum(self.x[:, self.k_Start_Port[k], :, k]) == 0 for k in range(self.goods_Display)]
        constraints += [np.sum(self.x[self.k_End_Port[k], :, :, k]) == 0 for k in range(self.goods_Display)]
        # 3.constraint for transition point
        for k in range(self.goods_Display):
            for j in range(self.port_Space):
                if (j != self.k_Start_Port[k]) & (j != self.k_End_Port[k]):
                    constraints.append(np.sum(self.x[:, j, :, k]) == np.sum(self.x[j, :, :, k]))
        # 4.each goods_Display can only be transitioned in or out of a port for at most once
        constraints += [np.sum(self.x[i, :, :, k]) <= 1 for k in range(self.goods_Display) for i in range(self.port_Space)]
        constraints += [np.sum(self.x[:, j, :, k]) <= 1 for k in range(self.goods_Display) for j in range(self.port_Space)]
        # 5.transition-out should be after transition-in
        constraints += [stayTime[j, k] >= 0 for j in range(self.port_Space) for k in range(self.goods_Display)]
        # 6.constraint for number of containers used
        numCtn = np.dot(self.x, self.k_Vol) / self.container_Vol
        constraints += [self.y[i, j, t] - numCtn[i, j, t] >= 0 \
                        for i in range(self.port_Space) for j in range(self.port_Space) for t in
                        range(self.date_Space) if not isinstance(self.y[i, j, t] - numCtn[i, j, t] >= 0, bool)]
        # 7. constraint to check whether a route is used
        constraints += [self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5) \
                        for i in range(self.port_Space) for j in range(self.port_Space) for t in
                        range(self.date_Space) if
                        not isinstance(self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5), bool)]
        # 8.time limitation constraint for each goods_Display
        constraints += [np.sum(arrTime[:, self.k_End_Port[k], :, k]) <= self.k_DDL[k] for k in range(self.goods_Display)
                        if not isinstance(np.sum(arrTime[:, self.k_End_Port[k], :, k]) <= self.k_DDL[k], bool)]
        model = cp.Problem(objective, constraints)

        self.objective = objective
        self.constraints = constraints
        self.model = model

    
    def solve_model(self, solver=cp.CBC):
        '''
        solve the optimization model & cache the optimized objective value, route and arrival time for each goods_Display.
        :param solver: the solver to use to solve the LP problem when framework is CVXPY, has no effect to the model
        Default solver is cvxpy.CBC, other open source solvers do not perform that well.
        :return: None
        '''
        try:
                self.objective_value = self.model.solve(solver)
                self.xs = np.zeros((self.port_Space, self.port_Space, self.date_Space, self.goods_Display))
                self.xs[self.var_location] = self.var.value
                self.ys = np.zeros((self.port_Space, self.port_Space, self.date_Space))
                self.ys[self.var_2_location] = self.var_2.value
                self.zs = np.zeros((self.port_Space, self.port_Space, self.date_Space))
                self.zs[self.var_3_location] = self.var_3.value

        except:
            raise Exception('Model is not solvable, no solution will be provided')

        nonzeroX = list(zip(*np.nonzero(self.xs)))
        nonzeroX = sorted(nonzeroX, key=lambda x: x[2])
        nonzeroX = sorted(nonzeroX, key=lambda x: x[3])
        nonzeroX = list(map(lambda x: (self.port_Index[x[0]], self.port_Index[x[1]], \
                                       (self.min_Date + pd.to_timedelta(x[2], unit='days')).date().isoformat(),
                                       x[3]), nonzeroX))

        self.wh_Cost_Final, arrTime, _ = self.warehouse_fee(self.xs)
        self.transport_Cost = np.sum(self.ys * self.tran_Cost) + np.sum(self.zs * self.tran_Fixed_Cost)
        self.tax_Cost = np.sum(self.tax_Pct * self.k_Value) + \
                       np.sum(np.sum(np.dot(self.xs, self.k_Value), axis=2) * self.transit_Duty)
        self.solution_ = {}
        self.arrTime_ = {}
        for i in range(self.goods_Display):
            self.solution_['goods_Display-' + str(i + 1)] = list(filter(lambda x: x[3] == i, nonzeroX))
            self.arrTime_['goods_Display-' + str(i + 1)] = (self.min_Date + pd.to_timedelta \
                (np.sum(arrTime[:, self.k_End_Port[i], :, i]), unit='days')).date().isoformat()

    def get_output_(self):
        '''After the model is solved, return total cost, final solution and arrival
        time for each of the goods_Display'''

        return self.objective_value, self.solution_, self.arrTime_

    def warehouse_fee(self, x):
        '''return warehouse fee, arrival time and stay time for each port.'''

        startTime = np.arange(self.date_Space).reshape(1, 1, self.date_Space, 1) * x
        arrTimeMtrx = startTime + self.tran_Time.reshape(self.port_Space, \
                                                        self.port_Space, self.date_Space, 1) * x
        arrTime = arrTimeMtrx.copy()
        arrTimeMtrx[:, self.k_End_Port.tolist(), :, range(self.goods_Display)] = 0
        stayTime = np.sum(startTime, axis=(1, 2)) - np.sum(arrTimeMtrx, axis=(0, 2))
        stayTime[self.k_Start_Port.tolist(), range(self.goods_Display)] -= self.k_Start_Time
        warehouseCost = np.sum(np.sum(stayTime * self.k_Vol, axis=1) * self.wh_Cost)

        return warehouseCost, arrTime, stayTime

    def txt_solution(self, route, order):
        '''transform the cached results to text.'''

        travelMode = dict(zip(zip(route['Source'], route['Destination']), route['Travel Mode']))
        txt = "Solution"
        txt += "\nNumber of goods_Display: " + str(order['Order Number'].count())
        txt += "\nTotal cost: " + str(self.transport_Cost + self.wh_Cost_Final + self.tax_Cost)
        txt += "\nTransportation cost: " + str(self.transport_Cost)
        txt += "\nWarehouse cost: " + str(self.wh_Cost_Final)
        txt += "\nTax cost: " + str(self.tax_Cost)

        for i in range(order.shape[0]):
            txt += "\n------------------------------------"
            txt += "\ngoods_Display-" + str(i + 1) + "  Category: " + order['Commodity'][i]
            txt += "\nStart date: " + pd.to_datetime(order['Order Date']) \
                .iloc[i].date().isoformat()
            txt += "\nArrival date: " + str(self.arrTime_['goods_Display-' + str(i + 1)])
            txt += "\nRoute:"
            solution = self.solution_['goods_Display-' + str(i + 1)]
            route_txt = ''
            a = 1
            for j in solution:
                route_txt += "\n(" + str(a) + ")Date: " + j[2]
                route_txt += "  From: " + j[0]
                route_txt += "  To: " + j[1]
                route_txt += "  By: " + travelMode[(j[0], j[1])]
                a += 1
            txt += route_txt

        return txt


def transform(filePath):
    '''Read in order and route data, transform the data into a form that can
    be processed by the operation research model.'''
    order = pd.read_excel(filePath, sheet_name='Order Information')
    route = pd.read_excel(filePath, sheet_name='Route Information')
    order['Tax Percentage'][order['Journey Type'] == 'Domestic'] = 0
    route['Cost'] = route[route.columns[7:12]].sum(axis=1)
    route['Time'] = np.ceil(route[route.columns[14:18]].sum(axis=1) / 24)
    route = route[list(route.columns[0:4]) + ['Fixed Freight Cost', 'Time', \
                                              'Cost', 'Warehouse Cost', 'Travel Mode', 'Transit Duty'] + list(
        route.columns[-9:-2])]
    route = pd.melt(route, id_vars=route.columns[0:10], value_vars=route.columns[-7:] \
                    , var_name='Weekday', value_name='Feasibility')
    route['Weekday'] = route['Weekday'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, \
                                                 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})

    return order, route

In [3]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
datapath = "content/drive/My Drive/Colab Notebooks/data/"

In [5]:
SAVEPATH = '/content/drive/My Drive/Colab Notebooks/data/'

In [6]:
if __name__ == '__main__':
    order, route = transform("/content/drive/My Drive/Colab Notebooks/data/model data.xlsx")
    m = SCO(framework='CVXPY') # for open source framework
    m.set_param(route, order)
    m.build_model()
    m.solve_model()
    txt = m.txt_solution(route, order)
    with open("/content/drive/My Drive/Colab Notebooks/data/Solution1.txt", "w") as text_file:
        text_file.write(txt)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [7]:
import folium

p = folium.Map(location=[40.720, -73.993],
              zoom_start=15)

loc = [(40.720, -73.993),
       (40.721, -73.996)]

folium.PolyLine(loc,
                color='red',
                weight=5,
                opacity=0.8).add_to(p)
p

In [8]:
route

,Route Number,Source,Destination,Container Size,Fixed Freight Cost,Time,Cost,Warehouse Cost,Travel Mode,Transit Duty,Weekday,Feasibility
0,1,Singapore Port,Shanghai Port,67,300,8.0,900,NaN,Sea,0.002,1,1
1,2,Shanghai Port,Singapore Port,67,150,8.0,900,NaN,Sea,0.001,1,1
2,3,Singapore Port,Malaysia Port,34,50,4.0,650,NaN,Sea,0.000,1,0
3,4,Malaysia Port,Singapore Port,34,50,4.0,650,NaN,Sea,0.001,1,0
4,5,Shanghai Port,Malaysia Port,67,300,7.0,730,NaN,Sea,0.000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
345,46,Shanghai Railway Station,Shanghai Warehouse,34,100,1.0,0,NaN,Truck,0.000,7,1
346,47,Singapore Warehouse,Malaysia Warehouse,34,150,1.0,0,20.0,Truck,0.000,7,1
347,48,Malaysia Warehouse,Singapore Warehouse,34,150,1.0,0,10.0,Truck,0.001,7,1
348,49,Shanghai Warehouse,Wuxi Warehouse,34,125,1.0,0,15.0,Truck,0.000,7,1


In [9]:
order

,Order Number,Ship From,Ship To,Commodity,Order Value,Weight (KG),Volume,Shipper Name,Shipper Address,Shipper Country,Consignee Country,Order Date,Required Delivery Date,Journey Type,Tax Percentage
0,1,Singapore Warehouse,Wuxi Warehouse,Honey,50000,21000,34,YCH,8 Bulim Ave,Singapore,China,2018-02-01,2018-02-25,International,0.03
1,2,Malaysia Warehouse,Shanghai Warehouse,Furniture,10000,20000,67,YCH,8 Bulim Ave,Malaysia,China,2018-02-02,2018-02-23,International,0.01
2,3,Singapore Warehouse,Shanghai Warehouse,Paper plates,12000,20000,67,YCH,8 Bulim Ave,Singapore,China,2018-02-03,2018-02-23,International,0.01
3,4,Singapore Warehouse,Shanghai Warehouse,Pharmaceutical drugs,800000,20,7,YCH,8 Bulim Ave,Singapore,China,2018-02-04,2018-02-24,International,0.10
4,5,Wuxi Warehouse,Malaysia Warehouse,Cigarette,700000,5000,50,YCH,8 Bulim Ave,China,Malaysia,2018-02-05,2018-02-22,International,0.15
5,6,Shanghai Warehouse,Singapore Warehouse,Apple,30000,25000,67,YCH,8 Bulim Ave,China,Singapore,2018-02-06,2018-02-23,International,0.01
6,7,Malaysia Warehouse,Singapore Warehouse,Durian,10000,10000,34,YCH,8 Bulim Ave,Malaysia,Singapore,2018-02-07,2018-02-24,International,0.01
7,8,Wuxi Warehouse,Shanghai Warehouse,Furniture,10000,20000,67,YCH,8 Bulim Ave,China,China,2018-02-08,2018-02-25,Domestic,0.00


In [10]:
m.get_output_()[1].get("goods_Display-1")

[('Singapore Warehouse', 'Malaysia Warehouse', '2018-02-01', 0),
 ('Malaysia Warehouse', 'Malaysia Port', '2018-02-02', 0),
 ('Malaysia Port', 'Shanghai Port', '2018-02-03', 0),
 ('Shanghai Port', 'Shanghai Warehouse', '2018-02-10', 0),
 ('Shanghai Warehouse', 'Wuxi Warehouse', '2018-02-11', 0)]

In [11]:
m.get_output_()[1].get("goods_Display-1")[0]

('Singapore Warehouse', 'Malaysia Warehouse', '2018-02-01', 0)

In [12]:
type(m.get_output_()[1].get("goods_Display-1")[0])

tuple

In [13]:
m.get_output_()[1].get("goods_Display-1")[0][0]

'Singapore Warehouse'